In [23]:
import pandas as pd
import time
import re
import datetime

In [18]:
import urllib.request
import urllib.parse
from bs4 import BeautifulSoup
from konlpy.tag import Hannanum
from tqdm.notebook import tqdm

In [19]:
# 수집
def search(name_list, ds, de):
    baseUrl = 'https://search.naver.com/search.naver?&where=news&query='
    url = baseUrl + urllib.parse.quote_plus(name_list) 
    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')

    subject = []
    date = []
    contents = []
    company = []
    
    page = 1
    
    while True:
        
        url = baseUrl + urllib.parse.quote_plus(name_list) + '&nso=so%3Ar%2Cp%3Afrom'+ str(ds) + 'to' + str(de) +'&start=' + str(page)
        html = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(html, 'html.parser')

        word = soup.find_all(class_='type01')[0]
        word = word.find_all('dl')

        title = soup.find_all(class_='_sp_each_title')

        com = soup.find_all(class_='_sp_each_source')

        date_re = re.compile(r'\d\d\d\d.\d\d.\d\d.')
        day = soup.find_all(class_='txt_inline')


        # 컨텐츠
        for words in word:
            contents.append(words.find_all('dd')[1].get_text())

        # 타이틀
        for tie in title:
            subject.append(tie.get_text())

        # 업체
        for coms in com:
            if coms.get_text()[-2:] == '선정':
                company.append(coms.get_text()[:-6])
            else:
                company.append(coms.get_text())

        # 시간
        for dates in day:
            try:
                match = date_re.search(dates.get_text())
                date.append(match.group())
            except:
                date_re2 = re.compile(r'\d')
                match = date_re2.search(dates.get_text())
                m = int(match.group())
                year = str(datetime.datetime.today().year)
                month = str(datetime.datetime.today().month)
                day = str(datetime.datetime.today().day-m)
                result = year + '.' + month + '.' + day + '.'
                date.append(result)
        
        print(page, end=' ')
        
        try:
            if soup.find_all(class_='next')[0].get_text() != '다음페이지':
                break
            else: 
                page += 10
        except:
            if soup.find(class_='next') != '다음페이지':
                break

    df = pd.DataFrame(list(zip(subject, date, contents, company)), columns = ['title', 'date','contents', 'company'])
    
    return df

In [20]:
# 전처리 2
def sentence(df):
    
    df['contents'] = df['contents'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]"," ")
    
    stopwords = ['을', '로', '의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다''의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

    hannanum = Hannanum()
    
    contents = []

    for sentence in tqdm(df['contents']):
        temp_x = []
        temp_x = hannanum.nouns(sentence)
        temp_x = [word for word in temp_x if not word in stopwords]
        contents.append(temp_x)
        
    return contents


In [21]:
# 단어 빈도수
def word_count(contents, name):
    wordlist = sum(contents, [])
    word_list = pd.Series(wordlist)
    result = word_list.value_counts()
    result.to_csv('data/기업 '+name+'.csv', mode='w')
    print(name + ' Save compelte')

In [6]:
result = pd.DataFrame(index=range(0,0), columns=['title', 'date','contents', 'company'])

In [7]:
com_list = pd.read_csv('data/com_list.csv')
com_list['ds'] = [date - 10000 for date in com_list['date']]
com_list.rename(columns = {'date' : 'de'}, inplace = True)

In [8]:
com_de = com_list['de'].tolist()
com_ds = com_list['ds'].tolist()
com_list = com_list['company'].tolist()
com_list = [cmp_lst.strip() for cmp_lst in company_list]

In [22]:
for cmp_lst, ds, de in zip(com_list, com_ds, com_de):
    print(cmp_lst + ' Start' + ' ds' + str(ds) +' de' + str(de))
    df = search(cmp_lst, ds, de)
    df_contents = sentence(df)
    word_count(df_contents, cmp_lst)
    result = pd.concat([result, df])

데코앤이 Start ds20181120 de20191120
1 11 21 31 41 51 61 71 81 91 101 111 121 131 141 151 161 171 181 191 201 211 221 231 241 251 261 271 281 291 301 311 321 331 341 351 361 371 381 391 401 411 421 431 441 451 461 471 481 491 501 511 521 531 541 551 561 571 581 591 601 611 621 631 641 651 661 671 681 691 701 711 721 731 741 751 761 771 781 791 801 811 821 831 841 851 861 871 881 891 901 911 921 931 941 951 961 971 981 991 1001 1011 1021 1031 1041 1051 1061 1071 1081 1091 1101 1111 1121 1131 1141 1151 1161 1171 1181 1191 1201 1211 1221 1231 1241 1251 1261 1271 1281 1291 1301 1311 1321 1331 1341 신텍 Start ds20170709 de20180709
1 11 21 31 41 51 61 71 81 91 101 111 121 131 141 151 161 171 181 191 201 211 221 231 241 251 261 271 281 291 301 311 321 331 341 351 361 371 381 391 401 411 421 431 441 451 461 471 481 491 501 511 521 531 541 551 561 571 581 591 601 611 621 631 641 651 661 671 681 691 701 711 721 731 741 751 761 771 781 791 801 811 821 831 841 851 861 871 881 891 901 911 921 931 941 951

KeyboardInterrupt: 

In [19]:
result.to_csv('data/네이버 기사.csv', mode='w', encoding='utf-8')

NameError: name 'result' is not defined